In [1]:
from auto_elm import AUTO_ELM 
#from os_MFB1 import MFB_ELM 
# Our proposed algorithm
import pandas as pd 
import math
import numpy as np
import tensorflow as tf
#from tqdm import tqdm_notebook as tqdm # Jupyter notebook should use this
import matplotlib.pyplot as plt
import os
import cv2
import scipy.io as scipy_io

train_dataset = np.array(pd.read_csv("UCI dataset/mushroom", header = None,delimiter=' '))
train_dataset.shape

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


(8124, 23)

In [2]:
label = train_dataset[:,0]
data = train_dataset[:,1:22]
label = label.reshape(label.shape[0], 1)
print(label)

tr_file = 'UCI dataset/mushroom'

[[1]
 [2]
 [2]
 ...
 [2]
 [1]
 [2]]


In [3]:
def process_data(fname, NUM_ATTR, NUM_CLASSES):
    I = np.eye(NUM_CLASSES)
    with open(fname) as file:
        xx = file.readlines()
        #print(xx)
        data = np.zeros([len(xx), NUM_ATTR])
        label = np.zeros(len(xx), dtype=int)
        label_onehot = []
        for i in range(len(xx)):
            tmp = xx[i].split(' ')
            label[i] = int(tmp[0])-1
            label_onehot.append(I[label[i]])
            for j in range(1,len(tmp)-1):
                position = int(tmp[j].split(':')[0])
                value = 1.0*int(tmp[j].split(':')[1])
                data[i][position-1] = value 
    return data, label, np.array(label_onehot, dtype=int)

In [4]:
tr_data, tr_label, tr_onehot = process_data(tr_file, NUM_ATTR=112, NUM_CLASSES=2)

In [5]:
y_train = tr_label[:4000]
y_test = tr_label[4000:]

In [6]:
tr_data = tr_data*0.01
x_train = tr_data[:4000]
x_test = tr_data[4000:]
y_train_onehot = tr_onehot[:4000]
y_test_onehot = tr_onehot[4000:]
print(x_train)
print(x_test.shape)
print(y_train_onehot.shape)
print(y_test_onehot.shape)

[[0.   0.   0.   ... 0.   0.01 0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.01 0.   ... 0.   0.   0.  ]
 ...
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.01 ... 0.   0.   0.  ]
 [0.   0.   0.01 ... 0.   0.   0.  ]]
(4124, 112)
(4000, 2)
(4124, 2)


In [12]:
border = 400
x_train_init = x_train[:border]
x_train_seq = x_train[border:]

print('total initial: ', (border))
print('total sequential: ', len(x_train_seq))
print('total testing: ', len(x_test))

total initial:  400
total sequential:  3600
total testing:  4124


In [13]:
n_input_nodes = 112
n_hidden_nodes = 37  # used to be 6
n_output_nodes = 112

In [14]:
tf.reset_default_graph()
# ===========================================
# Instantiate os-elm
# ===========================================
auto_elm = AUTO_ELM(
    # the number of input nodes.
    n_input_nodes=n_input_nodes,
    # the number of hidden nodes.
    n_hidden_nodes=n_hidden_nodes,
    # the number of output nodes.
    n_output_nodes=n_output_nodes,
    # loss function.
    # the default value is 'mean_squared_error'.
    # for the other functions, we support
    # 'mean_absolute_error', 'categorical_crossentropy', and 'binary_crossentropy'.
    c_value = 2.0**(-25),
    loss='mean_squared_error',
    # activation function applied to the hidden nodes.
    # the default value is 'sigmoid'.
    # for the other functions, we support 'linear' and 'tanh'.
    # NOTE: OS-ELM can apply an activation function only to the hidden nodes.
    activation='sin',
)

In [15]:
auto_elm.init_train(x_train_init, x_train_init)

import time

'''initial training evaluation'''
t1 = time.time()
[train_loss, accuracy] = auto_elm.evaluate(x_train_init, x_train_init, metrics=['loss', 'accuracy'])
[test_loss, accuracy] = auto_elm.evaluate(x_test, x_test, metrics=['loss', 'accuracy'])
print('train_loss: %f, train_accuracy: %f' % (train_loss, accuracy))
print('test_loss: %f, test_accuracy: %f' % (test_loss, accuracy))
t2 = time.time()

epoch_train_loss = []
epoch_test_loss = []

train_loss: 0.002967, train_accuracy: 0.000000
test_loss: 0.001453, test_accuracy: 0.000000


In [20]:
batch_size = 500


for epoch in range(100):
    #pbar = tqdm(total=len(x_train_seq), desc='sequential training phase')
    for i in range(0, len(x_train_seq), batch_size):
        x_batch = x_train_seq[i:i+batch_size]
        auto_elm.seq_train(x_batch, x_batch)
        #pbar.update(n=len(x_batch))
    '''epoch evaluation'''
    [train_loss, accuracy] = auto_elm.evaluate(x_train_init, x_train_init, metrics=['loss', 'accuracy'])
    [test_loss, accuracy] = auto_elm.evaluate(x_test, x_test, metrics=['loss', 'accuracy'])
    print('train_loss: %f, train_accuracy: %f' % (train_loss, accuracy))
    print('test_loss: %f, test_accuracy: %f' % (test_loss, accuracy))
    epoch_train_loss.append(train_loss)
    epoch_test_loss.append(test_loss)

    #pbar.close()

ValueError: Cannot feed value of shape (500, 37) for Tensor 'x:0', which has shape '(?, 112)'

In [17]:
x_train_encoded = 100*auto_elm.encoding(x_train)
x_test_encoded = 100*auto_elm.encoding(x_test)
#y_test = y_test_onehot
#y_train = y_train_onehot

print(x_train_encoded.shape)
print(x_test_encoded.shape)
print(y_train.shape)
print(y_test.shape)

border = 500
x_train_init = x_train_encoded[:border]
y_train_init = y_train[:border]
x_train_seq = x_train_encoded[border:]
y_train_seq = y_train[border:]

(4000, 37)
(4124, 37)
(4000,)
(4124,)


In [ ]:
n_input_nodes = 37
n_hidden_nodes = 1  # used to be 256
n_output_nodes = 2


In [ ]:
tf.reset_default_graph()
from os_MFB1 import MFB_ELM 

C = 4
mfb_elm = MFB_ELM(
    # the number of input nodes.
    n_input_nodes=n_input_nodes,
    # the number of hidden nodes.
    n_hidden_nodes=n_hidden_nodes,
    # the number of output nodes.
    n_output_nodes=n_output_nodes,
    #C = C,
    # loss function.
    # the default value is 'mean_squared_error'.
    # for the other functions, we support
    # 'mean_absolute_error', 'categorical_crossentropy', and 'binary_crossentropy'.
    loss='mean_squared_error',
    # activation function applied to the hidden nodes.
    # the default value is 'sigmoid'.
    # for the other functions, we support 'linear' and 'tanh'.
    # NOTE: OS-ELM can apply an activation function only to the hidden nodes.
    activation='sin',
    C = C
)

In [ ]:
mfb_elm.init_train(x_train_encoded, y_train_onehot)

'''initial training evaluation'''
[train_loss, train_accuracy] = mfb_elm.evaluate(x_train_encoded[:len(x_test_encoded)], y_train_onehot[:len(x_test_encoded)], metrics=['loss', 'accuracy'])
[test_loss, test_accuracy] = mfb_elm.evaluate(x_test_encoded, y_test_onehot, metrics=['loss', 'accuracy'])
print('train_loss: %f, train_accuracy: %f' % (train_loss, train_accuracy))
print('test_loss: %f, test_accuracy: %f' % (test_loss, test_accuracy))
    
epoch_train_accuracy = []
epoch_test_accuracy = []

In [ ]:
batch_size = 100


for epoch in range(10):
    #pbar = tqdm.tqdm(total=len(x_train), desc='sequential training phase')
    for i in range(0, len(x_train_seq), batch_size):
        x_batch = x_train_seq[i:i+batch_size]
        y_batch = y_train_seq[i:i+batch_size]
        if len(x_batch) != batch_size:
            break
        mfb_elm.seq_train(x_batch, y_batch)
       # pbar.update(n=len(x_batch))
    '''epoch evaluation'''
    [train_loss, train_accuracy] = mfb_elm.evaluate(x_train_encoded, y_train_onehot, metrics=['loss', 'accuracy'])
    [test_loss, test_accuracy] = mfb_elm.evaluate(x_test_encoded, y_test_onehot, metrics=['loss', 'accuracy'])
    print('train_loss: %f, train_accuracy: %f' % (train_loss, train_accuracy))
    print('test_loss: %f, test_accuracy: %f' % (test_loss, test_accuracy))
    epoch_train_accuracy.append(train_accuracy)
    epoch_test_accuracy.append(test_accuracy)

In [ ]:
from ELM import ELM 
tf.reset_default_graph()
# ===========================================
# Instantiate os-elm
# ===========================================

    
def one_hot(y, classes):
    I = np.eye(classes)
    ret = []
    for y_ in y:
        ret.append(I[y_])
    return np.array(ret)
    
#y_train_onehot = one_hot(y_train, 6)
#y_test_onehot = one_hot(y_test, 6)
    
model = ELM(n_input_nodes=37,
                    n_hidden_nodes=100,
                    n_output_nodes=2,
                    loss='mean_squared_error',
                    activation='sigmoid',
                    name='elm',
                    )
   
#feature_train = tr_data
#feature_test = ts_data
feature_train = auto_elm.encoding(x_train)
feature_test = auto_elm.encoding(x_test)

model.fit(feature_train, y_train_onehot)

test_loss, test_acc = model.evaluate(feature_test, y_test_onehot, metrics=['loss', 'accuracy'])
print('test_loss: %f' % test_loss)
print('test_acc: %f' % test_acc)
#accuracy_sum+=test_acc

pred = model.predict(x_test_encoded)
test_accuracy = np.sum(np.equal(np.argmax(pred,axis=1), np.argmax(y_test_onehot,axis=1)))/len(y_test_onehot)
print(test_accuracy)
print(np.argmax(pred[:100],axis=1))
print(np.argmax(y_test_onehot[:100],axis=1))

In [ ]:
np.random.seed(1337)  # for reproducibility
#from sklearn.datasets import load_digits
#from sklearn.model_selection import train_test_split
from sklearn.metrics.classification import accuracy_score

from dbn.tensorflow import SupervisedDBNClassification

classifier = SupervisedDBNClassification(hidden_layers_structure=[256, 256],
                                         learning_rate_rbm=0.05,
                                         learning_rate=0.01,
                                         n_epochs_rbm=10,
                                         n_iter_backprop=100,
                                         batch_size=32,
                                         activation_function='relu',
                                         dropout_p=0.2)
classifier.fit(x_train, y_train)
Y_pred = classifier.predict(x_test)
print('Done.\nAccuracy: %f' % accuracy_score(y_test, Y_pred))

In [ ]:
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(0.99)
pca.fit(x_train)
train_img = pca.transform(x_train)
test_img  = pca.transform(x_test)
train_img.shape,test_img.shape,x_train.shape

In [ ]:
from ELM import ELM 
tf.reset_default_graph()
# ===========================================
# Instantiate os-elm
# ===========================================

    
def one_hot(y, classes):
    I = np.eye(classes)
    ret = []
    for y_ in y:
        ret.append(I[y_])
    return np.array(ret)
    
#y_train_onehot = one_hot(y_train, 6)
#y_test_onehot = one_hot(y_test, 6)
    
model = ELM(n_input_nodes=37,
                    n_hidden_nodes=100,
                    n_output_nodes=2,
                    loss='mean_squared_error',
                    activation='sigmoid',
                    name='elm',
                    )
   
feature_train = train_img
feature_test = test_img

model.fit(feature_train, y_train_onehot)

test_loss, test_acc = model.evaluate(feature_test, y_test_onehot, metrics=['loss', 'accuracy'])
print('test_loss: %f' % test_loss)
print('test_acc: %f' % test_acc)
#accuracy_sum+=test_acc
   



    
print("==========================================")
print("OS-ELA ===================================")
print("==========================================")
#print("Average time: ", time_sum/10.0)
#print("Average accuracy: ", accuracy_sum/10.0)
print("==========================================")




        
            

In [ ]:
n_input_nodes = 37
n_hidden_nodes = 1  # used to be 256
n_output_nodes = 2

border = 100
x_train_init = feature_train[0:border]
y_train_init = y_train_onehot[0:border]
x_train_seq = feature_train[border:]
y_train_seq = y_train_onehot[border:]

In [ ]:
tf.reset_default_graph()
from os_MFB1 import MFB_ELM 
C = 7
mfb_elm = MFB_ELM(
    # the number of input nodes.
    n_input_nodes=n_input_nodes,
    # the number of hidden nodes.
    n_hidden_nodes=n_hidden_nodes,
    # the number of output nodes.
    n_output_nodes=n_output_nodes,
    #C = C,
    # loss function.
    # the default value is 'mean_squared_error'.
    # for the other functions, we support
    # 'mean_absolute_error', 'categorical_crossentropy', and 'binary_crossentropy'.
    loss='mean_squared_error',
    # activation function applied to the hidden nodes.
    # the default value is 'sigmoid'.
    # for the other functions, we support 'linear' and 'tanh'.
    # NOTE: OS-ELM can apply an activation function only to the hidden nodes.
    activation='sin',
    C = C
)

In [ ]:
mfb_elm.init_train(x_train_init, y_train_init)

'''initial training evaluation'''
[train_loss, train_accuracy] = mfb_elm.evaluate(feature_train[:len(feature_test)], y_train_onehot[:len(feature_test)], metrics=['loss', 'accuracy'])
[test_loss, test_accuracy] = mfb_elm.evaluate(feature_test, y_test_onehot, metrics=['loss', 'accuracy'])
print('train_loss: %f, train_accuracy: %f' % (train_loss, train_accuracy))
print('test_loss: %f, test_accuracy: %f' % (test_loss, test_accuracy))
    
epoch_train_accuracy = []
epoch_test_accuracy = []

In [ ]:
batch_size = 500


for epoch in range(10):
    #pbar = tqdm.tqdm(total=len(x_train), desc='sequential training phase')
    for i in range(0, len(x_train_seq), batch_size):
        x_batch = x_train_seq[i:i+batch_size]
        y_batch = y_train_seq[i:i+batch_size]
        if len(x_batch) != batch_size:
            break
        mfb_elm.seq_train(x_batch, y_batch)
       # pbar.update(n=len(x_batch))
    '''epoch evaluation'''
    [train_loss, train_accuracy] = mfb_elm.evaluate(feature_train, y_train_onehot, metrics=['loss', 'accuracy'])
    [test_loss, test_accuracy] = mfb_elm.evaluate(feature_test, y_test_onehot, metrics=['loss', 'accuracy'])
    print('train_loss: %f, train_accuracy: %f' % (train_loss, train_accuracy))
    print('test_loss: %f, test_accuracy: %f' % (test_loss, test_accuracy))
    epoch_train_accuracy.append(train_accuracy)
    epoch_test_accuracy.append(test_accuracy)